# Ansible によるサーバー操作

この演習では、先程起動した2台のサーバーに対して、Ansible を用いた設定の自動化を行っていきます。

![演習の概要](materials/image02-01.png "演習の概要")

## 鍵のアップロード

まずはじめに、aitac-automation-key.pem を Jupyter 上に登録します。Ansible はこの秘密鍵を使って操作対象のサーバーへとアクセスします。

鍵の登録は以下の手順で行ってください。

* Jupyter トップページの画面右の「Upload」を選択
* アップロードする「aitac-automation-key.pem」を選んでOK
* Jupyter 上に登録されたファイルの「Upload」ボタンを押下するとアップロードされます。

鍵の権限を設定しておきます。

In [1]:
# 実行例

chmod 400 aitac-automation-key.pem
ls -alF aitac-automation-key.pem

-r--------. 1 bit_kun users 1692 Jan  1 21:58 aitac-automation-key.pem


In [ ]:
# この Cell を実行してください。

chmod 400 aitac-automation-key.pem
ls -alF aitac-automation-key.pem

## インベントリの作成

今回の操作対象にするサーバーをインベントリへ登録します。

まず、AWSのダッシュボードから「bation」「ansible-taget」のIPv4パブリックIPを確認してメモしておいてください。

次に、以下のファイルを開いて例の様に編集します。

[inventories/02_server_ops](/edit/aitac-automation-handson/inventories/02_server_ops)

例）

    [bation]
    11.22.33.44 ← bation のIPアドレス
    
    [target]
    22.33.44.55 ← ansible-target 1台目のIPアドレス
    33.44.55.66 ← ansible-target 2台目のIPアドレス


## テスト接続の実行

ここまでは Playbook に記述したモジュールを ansible-playbook コマンドで実行してきました。

別の方法としてモジュールを直接 ansible から実行して動作テストを行うことが可能です。

### ansible-target サーバーへの接続テスト

以下は ping モジュールを使って、対象のホストへ接続可能かをテストしています。

In [4]:
# 実行例

ansible target -i inventories/02_server_ops -u centos --private-key aitac-automation-key.pem -b -m ping

13.230.56.76 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
13.115.202.69 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


オプションの意味

```
ansible \
    target \                                 # インベントリ内のグループを指定
    -i inventories/02_server_ops \           # インベントリファイルを指定
    -u centos \                              # 接続ユーザ
    --private-key aitac-automation-key.pem \ # 接続に使用する鍵ファイル
    -b \                                     # 接続後に root ユーザーへ
    -m ping                                  # 使用するモジュール
```

In [ ]:
# 実行してください。

ansible target -i inventories/02_server_ops -u centos --private-key aitac-automation-key.pem -b -m ping

### bation サーバーへの接続テスト

上記の例では、インベントリの中で作成した target というグループに属している2台のサーバーへの接続テストを行っています。

次の例のようにすると、bation のみへの実行が可能です。

In [ ]:
# 実行してください。

ansible bation -i inventories/02_server_ops -u centos --private-key aitac-automation-key.pem -b -m ping

### インベントリ内の全てのサーバーへの接続テスト

次の例では、all という特別なキーワードを使って、インベントリに記載されている全てのホストへの接続テストを行っています。

In [ ]:
# 実行してください。

ansible all -i inventories/02_server_ops -u centos --private-key aitac-automation-key.pem -b -m ping

このように、インベントリに複数のサーバーを記載して一括して操作を実行したり、グループを作成してそのグループのみに操作を行うことが可能です。

## ad-hoc 実行

上記のようにモジュールを直接してして実行する方法を Ansible ではad-hoc実行と呼びます。単純なモジュールであれば例のようにワンライナーのコマンドで実行が可能です。

以下は、 shell モジュールで任意のコマンドを実行する例になります。

In [ ]:
# 実行してください。

ansible all -i inventories/02_server_ops -u centos --private-key aitac-automation-key.pem -b -m shell -a 'hostname'

In [ ]:
# 実行してください。

ansible all -i inventories/02_server_ops -u centos --private-key aitac-automation-key.pem -b -m shell -a 'date'

In [ ]:
# 実行してください。

ansible all -i inventories/02_server_ops -u centos --private-key aitac-automation-key.pem -b -m shell -a 'df -h'

## HTTPDのセットアップ

ansible-target に対してHTTPDをインストールし有効化した上で、index.html を配置していきます。

ここで利用するモジュールは以下の3つになります。

* [yum モジュール](http://docs.ansible.com/ansible/latest/yum_module.html) パッケージをインストール・削除する
* [systemd モジュール](http://docs.ansible.com/ansible/latest/systemd_module.html) サービスを有効化・無効化する
* [template モジュール](http://docs.ansible.com/ansible/latest/template_module.html) ファイルを配置する
* import_role これは後に出てくるので、ここでは説明を省きます。出力されるログも無視しておいてください。

上記のモジュールを使った Playbook のサンプルが以下になります。

[server_ops_httpd_on.yml](/edit/aitac-automation-handson/server_ops_httpd_on.yml)

また、template モジュールは予め準備しておいたテンプレートファイルを対象サーバーへ配置する際に、テンプレート内部に記述された変数や処理を展開・実行してからサーバー上に配置します。

使用できる文法は [jinja2](http://docs.ansible.com/ansible/latest/playbooks_templating.html) 形式になります。

サンプルで利用するテンプレートファイルは以下になります。このファイルでは index.html.j2 内の変数が展開されてホストへ配布されます。

[index.html.j2](/edit/aitac-automation-handson/index.html.j2)

ではこのPlaybookを実行するとどうなるか確認してみましょう。

In [7]:
# 実行例

ansible-playbook -i inventories/02_server_ops -u centos --private-key aitac-automation-key.pem server_ops_httpd_on.yml


PLAY [target] ******************************************************************

TASK [Gathering Facts] *********************************************************
ok: [13.230.56.76]
ok: [13.115.202.69]

TASK [yum] *********************************************************************
changed: [13.115.202.69]
changed: [13.230.56.76]

TASK [systemd] *****************************************************************
changed: [13.115.202.69]
changed: [13.230.56.76]

TASK [template] ****************************************************************
changed: [13.115.202.69]
changed: [13.230.56.76]

PLAY RECAP *********************************************************************
13.115.202.69              : ok=4    changed=3    unreachable=0    failed=0   
13.230.56.76               : ok=4    changed=3    unreachable=0    failed=0   



In [ ]:
# 実行してください

ansible-playbook -i inventories/02_server_ops -u centos --private-key aitac-automation-key.pem server_ops_httpd_on.yml

実行したら、それぞれのIPアドレスへブラウザでアクセスしてみてください。設定されたトップページが参照できるはずです。

もしくは以下の CELL を実行するとそれぞれのサーバーの index.html を取得してくれます。

In [ ]:
# 実行してください。

ansible target -i inventories/02_server_ops -u centos --private-key aitac-automation-key.pem -b \
               -m shell -a 'curl -s localhost'

Ansible を利用すると、このように自動化の処理を記述することができます。

また、今作成した [server_ops_httpd_on.yml](/edit/aitac-automation-handson/server_ops_httpd_on.yml) は、先頭の hosts 部分で target グループを指定しています。この部分を書き換えることで、実行先を切り替えることも可能です。

更に -i で指定しているインベントリファイルを書き換えることで、全く別のサーバー群に対しても処理を実行することができます。

このように、Ansible では、何を実行するか（Playbook）、どこに対して実行するか（Inventory）を分けて管理することで、再利用性の高い自動化を実現することが可能です（Playbookの再利用に関しては後の演習で出てきます）

## Ansible の冪等性（べきとうせい）

ここまでの演習を振り返って、こう思った方もいらっしゃると思います。

「こんなのシェルスクリプトを書くのと何が違うんだ？」

単純な処理の実行だけであれば、わざわざシェルスクリプでできるものを Playbook にする必要はありません。では、Ansible を使うメリットは何なのかと言うと、この節のタイトルになっている「冪等性」が大きなメリットになります。

まず、先程実行したHTTPDを有効化するPlaybookをもう一度実行してみてください。

処理する内容は同じですが、先程は設定前の状態で合ったのに対して、今回は全て設定済みの状態でPlaybookを実行することになります。

In [10]:
# 実行例

ansible-playbook -i inventories/02_server_ops -u centos --private-key aitac-automation-key.pem server_ops_httpd_on.yml


PLAY [target] ******************************************************************

TASK [Gathering Facts] *********************************************************
ok: [13.115.202.69]
ok: [13.230.56.76]

TASK [yum] *********************************************************************
ok: [13.115.202.69]
ok: [13.230.56.76]

TASK [systemd] *****************************************************************
ok: [13.230.56.76]
ok: [13.115.202.69]

TASK [template] ****************************************************************
ok: [13.115.202.69]
ok: [13.230.56.76]

PLAY RECAP *********************************************************************
13.115.202.69              : ok=4    changed=0    unreachable=0    failed=0   
13.230.56.76               : ok=4    changed=0    unreachable=0    failed=0   



In [ ]:
# 実行してください

ansible-playbook -i inventories/02_server_ops -u centos --private-key aitac-automation-key.pem server_ops_httpd_on.yml

先程とは実行時に出力されるログに違いがあることに気づくと思います(set_fast_repo部分は一旦無視してください)

1回目と2回目で異なる箇所は、それぞれの処理において「changed」と出力されたか、「ok」と出力されたかになります。

* changed ・・・ Ansibleが処理を実行した結果、対象ホストの状態が変わった（Ansibleが実際に設定を行った）
* ok ・・・ Ansibleは処理を行おうとしたが、既に想定した設定になっているので状態が変わらなかった（Ansibleは設定を行わなかった・行う必要がなかった）

これが、Ansible が備える冪等性になります。つまり、Ansible は今から行おうとする処理を実行する必要があるのか、無いのかを実行前に判断してくれています。

では、この冪等性は何が嬉しいのかというと、

* Playbookには「処理の手順ではなく、状態を宣言的に記述できる」→ Playbook＝設定パラメータ＋手順書となる。
* 仮に複数台のホストに対して実行した処理が途中で失敗しても、最初から流し直せる（設定が成功した部分はスキップされるため）

Ansible の各モジュールはこの冪等性を考慮するように作られており、このモジュールを利用することで簡単に、そして安全に自動化を記述することができるようになっています。

これがスクリプトの場合、特に再実行においてはスクリプトを頭から流し直していいのか？ダメなのか？等の面倒な考慮点が生まれてしまうことは容易に想像できると思います。

*※ただし、Ansibleも全てのモジュールが完全な冪等性を保証しているわけではありません。モジュールの中には shell のように何が実行されるかわからないものや、操作対象（NW系機器やクラウド環境）によっては原理的に冪等性の確保が難しいものも存在しています。こういったモジュールを使う場合は利用者が注意を払う必要があります*

本演習はここまでです。